**Generate Synthetic data**

In [ ]:
import pandas as pd
import numpy as np

def generate_clean_data():
    # Generating synthetic sales data
    np.random.seed(42)
    start_date = pd.to_datetime('2019-01-01')
    end_date = pd.to_datetime('2022-12-31')
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')

    sales_data = pd.DataFrame({
        'Date': date_range,
        'ProductID': np.random.randint(1, 101, size=len(date_range)),
        'Region': np.random.choice(['North', 'South', 'East', 'West'], size=len(date_range)),
        'QuantitySold': np.random.randint(10, 100, size=len(date_range))
    })

    # Generating synthetic external factors data
    external_factors_data = pd.DataFrame({
        'Date': date_range,
        'EconomicIndicator': np.random.uniform(0.8, 1.2, size=len(date_range)),
        'WeatherImpact': np.random.uniform(0.5, 1.5, size=len(date_range))
    })

    # Generating synthetic supplier data
    supplier_data = pd.DataFrame({
        'SupplierID': np.arange(1, 51),
        'SupplierName': [f'Supplier_{i}' for i in range(1, 51)],
        'ReliabilityScore': np.random.uniform(0.7, 1.0, size=50),
        'CostEffectivenessScore': np.random.uniform(0.5, 1.0, size=50)
    })

    # Save to CSV
    sales_data.to_csv('historical_sales_data.csv', index=False)
    external_factors_data.to_csv('external_factors_data.csv', index=False)
    supplier_data.to_csv('supplier_data.csv', index=False)

    return sales_data, external_factors_data, supplier_data

def main():
    sales_data, external_factors_data, supplier_data = generate_clean_data()

    # Displaying the first few rows of each generated dataset
    print("Historical Sales Data:")
    print(sales_data.head())

    print("\nExternal Factors Data:")
    print(external_factors_data.head())

    print("\nSupplier Data:")
    print(supplier_data.head())

if __name__ == "__main__":
    main()


# Dashboard app

In [ ]:
!pip install adbench
!pip install --upgrade adbench

In [ ]:
import gradio as gr
import pandas as pd
import numpy as np

# Load synthetic data for demonstration
sales_data = pd.read_csv('historical_sales_data.csv')
external_factors_data = pd.read_csv('external_factors_data.csv')
supplier_data = pd.read_csv('supplier_data.csv')

# Function to simulate real-time monitoring and generate alerts
def real_time_monitoring(product_id, region):
    # Simulate data retrieval based on product_id and region
    product_sales = sales_data[(sales_data['ProductID'] == int(product_id)) & (sales_data['Region'] == region)]
    external_factors = external_factors_data[external_factors_data['Date'] == pd.to_datetime('today').strftime('%Y-%m-%d')]
    supplier_info = supplier_data.sample()

    # Calculate key metrics
    total_sales = product_sales['QuantitySold'].sum()
    economic_indicator = external_factors['EconomicIndicator'].values[0]
    supplier_reliability = supplier_info['ReliabilityScore'].values[0]
    supplier_cost_effectiveness = supplier_info['CostEffectivenessScore'].values[0]

    # Define alert conditions
    alert_conditions = {
        'LowSales': total_sales < 50,
        'HighEconomicIndicator': economic_indicator > 1.2,
        'LowSupplierReliability': supplier_reliability < 0.8,
        'HighSupplierCost': supplier_cost_effectiveness < 0.5
    }

    # Generate alerts based on conditions
    alerts = {key: "Alert!" for key, condition in alert_conditions.items() if condition}

    # Build dashboard display
    dashboard_display = f"""
    <h2>Real-Time Monitoring Dashboard</h2>
    <p>Product ID: {product_id}</p>
    <p>Region: {region}</p>
    <p>Total Sales: {total_sales}</p>
    <p>Economic Indicator: {economic_indicator}</p>
    <p>Supplier Reliability: {supplier_reliability}</p>
    <p>Supplier Cost Effectiveness: {supplier_cost_effectiveness}</p>
    <h3>Alerts</h3>
    <ul>
    """

    # Add alerts to display
    for alert, message in alerts.items():
        dashboard_display += f"<li>{alert}: {message}</li>"

    dashboard_display += "</ul>"

    return dashboard_display

# Gradio Interface
iface_real_time_monitoring = gr.Interface(
    fn=real_time_monitoring,
    inputs=["text", gr.Radio(['North', 'South', 'East', 'West'], label="Region")],
    outputs=gr.Textbox(),
    live=True,
    title="Real-Time Monitoring Dashboard"
)

# Launch the Gradio app
iface_real_time_monitoring.launch()


**bold text**## Inventory Managment

In [ ]:
import gradio as gr
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

# Load historical sales data
sales_data = pd.read_csv('historical_sales_data.csv')

# Feature engineering: Extracting features and target variable
X = sales_data[['ProductID', 'Region']]
y = sales_data['QuantitySold']

# Convert categorical variables to dummy/indicator variables
encoder = OneHotEncoder(drop='first')
X_encoded = encoder.fit_transform(X[['Region']])
X_encoded_df = pd.DataFrame(X_encoded.toarray(), columns=encoder.get_feature_names(['Region']))
X = pd.concat([X[['ProductID']], X_encoded_df], axis=1)

# Initialize and train a linear regression model
model = Pipeline([
    ('encoder', OneHotEncoder(drop='first')),
    ('regressor', LinearRegression())
])

model.fit(X, y)

# Function to calculate cost savings
def calculate_cost_savings(product_id, region, current_inventory, lead_time):
    # Predict quantity sold using the AI model
    input_data = pd.DataFrame({'ProductID': [product_id], 'Region': [region]})
    input_data_encoded = encoder.transform(input_data[['Region']])
    input_data_encoded_df = pd.DataFrame(input_data_encoded.toarray(), columns=encoder.get_feature_names(['Region']))
    input_data = pd.concat([input_data[['ProductID']], input_data_encoded_df], axis=1)
    predicted_quantity = int(model.predict(input_data)[0])

    # Calculate cost savings
    current_demand = predicted_quantity
    optimized_demand = int(model.predict(input_data)[0])
    holding_cost_current = current_inventory * 1  # Assuming holding cost per unit is 1
    holding_cost_optimized = min(current_inventory, optimized_demand * lead_time) * 1
    cost_savings = holding_cost_current - holding_cost_optimized

    return f"Cost Savings: ${cost_savings}"

# Gradio Interface
iface = gr.Interface(
    fn=calculate_cost_savings,
    inputs=["text", gr.Radio(['North', 'South', 'East', 'West'], label="Region"), "text", "text"],
    outputs="text",
    live=True,
    title="Inventory Management System"
)

# Launch the Gradio app
iface.launch()


**bold text**## Supplier Collaboration

In [ ]:
import gradio as gr
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Load synthetic supplier data
supplier_data = pd.read_csv('supplier_data.csv')

# Feature engineering: Extracting features and target variables
X = supplier_data[['SupplierID']]
y_reliability = supplier_data['ReliabilityScore']
y_cost_effectiveness = supplier_data['CostEffectivenessScore']

# Initialize and train a linear regression model for reliability score
reliability_model = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

reliability_model.fit(X, y_reliability)

# Initialize and train a linear regression model for cost-effectiveness score
cost_effectiveness_model = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

cost_effectiveness_model.fit(X, y_cost_effectiveness)

# Function to simulate supplier collaboration and negotiation
def supplier_collaboration(supplier_id):
    reliability_score = reliability_model.predict([[supplier_id]])[0]
    cost_effectiveness_score = cost_effectiveness_model.predict([[supplier_id]])[0]

    # Simulating negotiation logic
    negotiated_reliability = min(1, reliability_score + 0.1)  # Cap reliability increase at 10%
    negotiated_cost_effectiveness = max(0, cost_effectiveness_score - 0.1)  # Reduce cost-effectiveness by 10%

    return f"Supplier ID: {supplier_id}\nNegotiated Reliability Score: {negotiated_reliability:.2f}\nNegotiated Cost-Effectiveness Score: {negotiated_cost_effectiveness:.2f}"

# Gradio Interface
iface_supplier_collaboration = gr.Interface(
    fn=supplier_collaboration,
    inputs="text",
    outputs="text",
    live=True,
    title="Supplier Collaboration Showcase"
)

# Launch the Gradio app
iface_supplier_collaboration.launch()
